In [18]:
import os
import vllm
import json
from easysteer.hidden_states import get_all_hidden_states
from vllm import LLM

from easysteer.steer import extract_pca_control_vector,StatisticalControlVector


os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["VLLM_USE_V1"] = "0"

file_path = "../../temp/test.jsonl" #MATH-500
# file_path = "../../temp/test_gsm8k.jsonl" #GSM8K

model_path = "/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/"

# vector_path = "vectors/thinking_switch_pca_MATH-500.gguf" #MATH-500
vector_path = "MATH500.gguf"

num_question = 10

In [19]:
problem_list = []

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        line_count = 0
        for line in f:
            if line_count >= num_question:
                break
            stripped_line = line.strip()
            if not stripped_line:
                continue
                
            try:
                data = json.loads(stripped_line)
                if "problem" in data:
                    problem_list.append(data["problem"])
                    line_count += 1
                elif "question" in data:
                    problem_list.append(data["question"])
                    line_count += 1
            except json.JSONDecodeError:
                print(f"skip: {line[:50]}...")
                continue

except Exception as e:
    print(f"{str(e)}")
# problem_list = ["Find the roots of $(x - 3)^3 + (x -7)^3 = (2x - 10)^3.$","A regular hexagon can be divided into six equilateral triangles. If the perimeter of one of the triangles is 21 inches, what is the perimeter, in inches, of the regular hexagon?",""]

In [20]:
slow_thinking = [
    "Please give the detailed thinking progress.",
    "Output the complete thinking steps.",
    "Walk me through your reasoning process step by step.",
    "Show me your detailed analysis and thought process.",
    "Please break down your thinking into clear steps.",
    "I want to see your complete reasoning chain.",
    "Explain your approach and methodology thoroughly.",
    "Provide a comprehensive breakdown of your solution process.",
    "Show me how you arrived at this conclusion with full details.",
    "Give me the complete analytical framework you used."
]

fast_thinking = [
    "Output the answer directly.",
    "Just give me the answer.",
    "Skip the explanation, just provide the result.",
    "Cut to the chase - what's the answer?",
    "Give me the bottom line immediately.",
    "No need for details, just the final answer.",
    "Straight to the point please.",
    "Just the conclusion, no process needed.",
    "Direct answer only.",
    "Quick response - answer first."
]

len_slow=len(slow_thinking)
len_fast=len(fast_thinking)

In [21]:
texts = []
for prob in problem_list:
    for req in slow_thinking+fast_thinking:
        texts.append(f"""<|im_start|>user
{prob}{req}<|im_end|>
<|im_start|>assistant
That""")

llm = LLM(model=model_path,task="reward",tensor_parallel_size=1)

all_hidden_states, outputs = get_all_hidden_states(llm, texts)


INFO 07-12 16:53:31 [config.py:1472] Using max model len 32768
INFO 07-12 16:53:31 [config.py:4615] Only "last" pooling supports chunked prefill and prefix caching; disabling both.
INFO 07-12 16:53:31 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294.d20250712) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend='')

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.05it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.05it/s]


INFO 07-12 16:53:34 [default_loader.py:272] Loading weights took 1.05 seconds


INFO 07-12 16:53:34 [model_runner.py:1255] Model loading took 2.8793 GiB and 1.140728 seconds


Processed prompts: 100%|██████████| 200/200 [00:02<00:00, 93.55it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


In [22]:

control_vector = extract_pca_control_vector(
    all_hidden_states=all_hidden_states,
    positive_indices=list(range(num_question*len_slow)), 
    negative_indices=list(range(num_question*len_slow,num_question*(len_slow+len_fast))),
    model_type="qwen2.5",
    token_pos=-1,
    normalize=True
)

Computing PCA directions: 100%|██████████| 28/28 [00:00<00:00, 289.85it/s]


In [23]:
control_vector.export_gguf(vector_path)
control_vector = StatisticalControlVector.import_gguf(vector_path)
print(control_vector)

StatisticalControlVector(model_type='qwen2.5', method='pca', directions={0: memmap([ 0.00368822, -0.01995319,  0.00795989, ...,  0.02261038,
         0.01058245,  0.01832754], shape=(1536,), dtype=float32), 1: memmap([ 0.01206535,  0.01387033,  0.01741896, ...,  0.01243861,
        -0.01074592, -0.00290922], shape=(1536,), dtype=float32), 2: memmap([ 0.01887226, -0.01097931,  0.03007578, ..., -0.03247849,
        -0.0014588 ,  0.02354636], shape=(1536,), dtype=float32), 3: memmap([-0.00660873,  0.01031956, -0.00936981, ...,  0.03917792,
        -0.01935721,  0.02405123], shape=(1536,), dtype=float32), 4: memmap([-0.00518945, -0.03150864, -0.00232761, ...,  0.03491148,
        -0.01083705,  0.01022366], shape=(1536,), dtype=float32), 5: memmap([ 0.00250724, -0.03294224, -0.00282546, ...,  0.0102722 ,
        -0.07062968, -0.00541657], shape=(1536,), dtype=float32), 6: memmap([-0.00243617,  0.03178823,  0.0060613 , ...,  0.01789014,
        -0.03728993, -0.01727501], shape=(1536,), dtype

: 